# Monthly Landsat DSWE Generator

This code employs Landsat 5, 7, 8, and 9 remote sensing data within the Dynamic Surface Water Extent (DSWE) algorithm to develop monthly water inundation extent maps and supplemental gridded products (QC raster and RGB of source composite) for a given study area. The code first creates monthly composites from avaialable Landsat data, then applies the algorithm. If the total study area has <95% coverage (accounting for cloud and cloud shadowed pixels), the non-classified areas are classified using a composite of Landsat imagery from +- 1 month. If the coverage is still < 95%, the remaining unclassifed area is classified using a range of +- 2 months and so on. The maximum expansion range default is +- 3 months, but may be edited. The resulting exports (GEE assets) include the DSWE product, the RGB bands of the source composite, and a QC product showing the total expansion range needed to classify each pixel, with 0 being the highest quality and 3 being the lowest.

DSWE Methodology: Jones, J.W., 2019. Improved Automated Detection of Subpixel-Scale Inundation—Revised Dynamic Surface Water Extent (DSWE) Partial Surface Water Tests. Remote Sensing 11, 374. https://doi.org/10.3390/rs11040374

Landsat Collection 2: Earth Resources Observation and Science (EROS) Center. (2020). Landsat 8-9 Operational Land Imager / Thermal Infrared Sensor Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9OGBGM6. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 7 Enhanced Thematic Mapper Plus Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9C7I13B. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 4-5 Thematic Mapper Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9IAXOVV.

Google Earth Engine: Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., Moore, R., 2017. Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote Sensing of Environment, Big Remotely Sensed Data: tools, applications and experiences 202, 18–27. https://doi.org/10.1016/j.rse.2017.06.031

Code adapted from Dr. Evan Greenberg: https://github.com/evan-greenbrg Greenberg, E., Chadwick, A.J., Ganti, V., 2023. A Generalized Area-Based Framework to Quantify River Mobility From Remotely Sensed Imagery. Journal of Geophysical Research: Earth Surface 128, e2023JF007189. https://doi.org/10.1029/2023JF007189

Author: James (Huck) Rees, PhD Student, UC Santa Barbara Geography

Date: August 18, 2025

## Import packages

In [2]:
import ee
import geopandas as gpd
import os
import logging
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar

ee.Initialize()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Initialize functions

In [3]:
# Normalized Difference Water Index (MNDWI)
def Mndwi(image):
    """
    Calculate the Modified Normalized Difference Water Index (MNDWI) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MNDWI band named 'mndwi'.
    """
    return image.normalizedDifference(['Green', 'Swir1']).rename('mndwi')

# Modified Bare Soil Reflectance Variables
def Mbsrv(image):
    """
    Calculate the Modified Bare Soil Reflectance Variable (MBSRV) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MBSRV band named 'mbsrv'.
    """
    return image.select(['Green']).add(image.select(['Red'])).rename('mbsrv')

def Mbsrn(image):
    """
    Calculate the Modified Bare Soil Reflectance Normalized (MBSRN) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MBSRN band named 'mbsrn'.
    """
    return image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn')

# Normalized Difference Vegetation Index (NDVI)
def Ndvi(image):
    """
    Calculate the Normalized Difference Vegetation Index (NDVI) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the NDVI band named 'ndvi'.
    """
    return image.normalizedDifference(['Nir', 'Red']).rename('ndvi')

# Automated Water Extraction Index (AWESH)
def Awesh(image):
    """
    Calculate the Automated Water Extraction Index (AWEsh) for a given image.

    Parameters:
    image (ee.Image): The input image with the necessary bands for MBSRN calculation.

    Returns:
    ee.Image: The resulting image with the AWEsh band named 'awesh'.
    """
    return image.expression(
        'Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2',
        {
            'Blue': image.select(['Blue']),
            'Green': image.select(['Green']),
            'mbsrn': Mbsrn(image).select(['mbsrn']),
            'Swir2': image.select(['Swir2'])
        }
    ).rename('awesh')

# Decision Tree for Surface Water Extent (DSWE)
def Dswe(image):
    """
    Calculate the Decision Tree for Surface Water Extent (DSWE) for a given image.

    Parameters:
    image (ee.Image): The input image with bands required for the DSWE calculation.

    Returns:
    ee.Image: The resulting image with the DSWE classification named 'dswe'.
    """
    mndwi = Mndwi(image)
    mbsrv = Mbsrv(image)
    mbsrn = Mbsrn(image)
    awesh = Awesh(image)
    swir1 = image.select(['Swir1'])
    nir = image.select(['Nir'])
    ndvi = Ndvi(image)
    blue = image.select(['Blue'])
    swir2 = image.select(['Swir2'])

    # Decision tree thresholds
    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = (mndwi.gt(-0.44)
          .And(swir1.lt(0.09))
          .And(nir.lt(0.15))
          .And(ndvi.lt(0.7)))
    t5 = (mndwi.gt(-0.5)
          .And(blue.lt(0.1))
          .And(swir1.lt(0.3))
          .And(swir2.lt(0.1))
          .And(nir.lt(0.25)))

    # Combine results using weights to create unique classes
    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    # Define DSWE classification levels
    noWater = t.eq(0).Or(t.eq(1)).Or(t.eq(10)).Or(t.eq(100)).Or(t.eq(1000))
    hWater = t.eq(1111).Or(t.eq(10111)).Or(t.eq(11011)).Or(t.eq(11101)).Or(t.eq(11110)).Or(t.eq(11111))
    mWater = (t.eq(111).Or(t.eq(1011)).Or(t.eq(1101)).Or(t.eq(1110))
              .Or(t.eq(10011)).Or(t.eq(10101)).Or(t.eq(10110))
              .Or(t.eq(11001)).Or(t.eq(11010)).Or(t.eq(11100)))
    pWetland = t.eq(11000)
    lWater = (t.eq(11).Or(t.eq(101)).Or(t.eq(110)).Or(t.eq(1001))
              .Or(t.eq(1010)).Or(t.eq(1100)).Or(t.eq(10000))
              .Or(t.eq(10001)).Or(t.eq(10010)).Or(t.eq(10100)))

    # Assign classification levels to DSWE
    iDswe = (noWater.multiply(0)
             .add(hWater.multiply(4))
             .add(mWater.multiply(3))
             .add(pWetland.multiply(2))
             .add(lWater.multiply(1)))

    return iDswe.rename(['dswe'])

def maskL8sr(image):
    """
    Masks out clouds and cloud shadows in Landsat 8/9 imagery using the BQA band.
    Uses bits 8–9 for cloud confidence and bits 10–11 for shadow confidence.

    Args:
        image: ee.Image, the input Landsat image.

    Returns:
        ee.Image: The masked image with cloud and shadow pixels removed.
    """
    qa = image.select('BQA')

    # Cloud confidence (bits 8–9): mask if medium or high
    cloud_conf = qa.rightShift(8).bitwiseAnd(3)
    cloud_ok = cloud_conf.lte(1)

    # Shadow confidence (bits 10–11): mask if medium or high
    shadow_conf = qa.rightShift(10).bitwiseAnd(3)
    shadow_ok = shadow_conf.lte(1)

    mask = cloud_ok.And(shadow_ok)
    return image.updateMask(mask)

def getLandsatCollection():
    """
    Merges Landsat 5, 7, 8, and 9 collections (Tier 1, Collection 2 SR) 
    and standardizes the band names for consistent analysis.

    Returns:
        ee.ImageCollection: A merged collection of standardized Landsat images.
    """
    # Define the band mappings for each Landsat version
    bn9 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
    bn8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
    bn7 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    bn5 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    # Standardized names for all bands
    standard_bands = ['uBlue', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'BQA']

    # Fetch and rename bands in the Landsat collections
    ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").select(bn5, standard_bands)
    ls7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterDate('1999-04-15', '2003-05-30').select(bn7, standard_bands)
    ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(bn8, standard_bands)
    ls9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").select(bn9, standard_bands)

    # Merge all collections
    merged_collection = ls5.merge(ls7).merge(ls8).merge(ls9)

    return merged_collection

def rescale(image):
    """
    Rescale the reflectance values of Landsat imagery to allow for use of Landsat Collection 2 in DSWE.

    Parameters:
    image (ee.Image): The input image with bands to be rescaled.

    Returns:
    ee.Image: The image with rescaled bands added.
    """
    bns = ['uBlue', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2']
    optical_bands = image.select(bns).multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, None, True)

def load_roi(shapefile_path):
    """Load ROI from a shapefile and return as an EE Geometry."""
    gdf = gpd.read_file(shapefile_path)
    return ee.Geometry.Polygon(gdf.unary_union.__geo_interface__["coordinates"])

# Function to generate a water mask for a given year and polygon feature
def get_water_mask_for_feature(start_date, end_date, max_level, polygon):
    """
    Generate a water mask for a given year and polygon feature using Landsat imagery.

    Parameters:
    year (int): The year for which to generate the water mask.
    max_level (float): The maximum DSWE water level threshold for classification.
    polygon (ee.Geometry.Polygon): The polygon feature defining the area of interest.

    Returns:
    ee.Image: The water mask image for the specified year and polygon.
    """
    imagery = (getLandsatCollection()
               .map(maskL8sr)
               .map(rescale)
               .filterDate(start_date, end_date)
               .filterBounds(polygon))

    image_composite = imagery.median().clip(polygon)
    water_mask = Dswe(image_composite)

    return water_mask

def export_to_asset(image, year, month, roi, asset_folder, bands=None, product_name="DSWE"):
    """
    Export a single-band or multi-band Earth Engine image to a GEE asset, 
    with proper metadata and asset ID formatting.
    
    Parameters:
        image (ee.Image): The image to export
        year (int): The image year
        month (int): The image month
        roi (ee.Geometry): Region of interest for clipping/export
        asset_folder (str): GEE asset folder path (no trailing slash)
        bands (list of str): Specific bands to export
        product_name (str): Prefix for the image asset name (e.g. "DSWE", "QC", "Composite")
    """
    asset_id = f"{asset_folder}/{product_name}_{year}_{month:02d}"
    
    # Select only specified bands if provided
    if bands:
        image_to_export = image.select(bands)
    else:
        image_to_export = image

    # Compute last day of month
    last_day = calendar.monthrange(year, month)[1]

    # Prepare image metadata
    image_with_metadata = image_to_export.set({
        'system:time_start': ee.Date(f"{year}-{month:02d}-01").millis(),
        'system:time_end': ee.Date(f"{year}-{month:02d}-{last_day:02d}").millis(),
        'source_ids': image.get('source_ids')
    })

    # Check if the asset already exists
    try:
        ee.data.getAsset(asset_id)
        logging.info(f"Skipping {asset_id}, already exists.")
    except:
        # Export the image with metadata
        task = ee.batch.Export.image.toAsset(
            image=image_with_metadata,
            description=f"{product_name}_{year}_{month:02d}",
            assetId=asset_id,
            scale=30,
            region=roi,
            maxPixels=1e13
        )
        task.start()
        logging.info(f"Exporting {asset_id}...")
        
def get_filled_composite_before_dswe(start_date, end_date, roi, max_months=6, fill_threshold=0.95):
    def get_composite(s, e):
        collection = (getLandsatCollection()
                      .map(maskL8sr)
                      .map(rescale)
                      .filterDate(s, e)
                      .filterBounds(roi))

        ids = collection.aggregate_array('system:index').distinct().sort()
        id_string = ids.join(",")  # Convert to comma-separated string
        return collection.median().clip(roi).set({'source_ids': id_string})

    def get_coverage(image):
        valid = image.select("Green").mask()
        stats = valid.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=30,
            maxPixels=1e13
        )
        return ee.Number(stats.get("Green"))

    base = get_composite(start_date, end_date)
    last_used_composite = base

    filled = base
    expansion_mask = base.select("Green").mask().Not().multiply(0).rename("expansion_mask")
    coverage = get_coverage(filled)

    for offset in range(1, max_months + 1):
        if coverage.gte(fill_threshold).getInfo():
            break

        delta = relativedelta(months=offset)
        expanded_start = (datetime.strptime(start_date, "%Y-%m-%d") - delta).strftime("%Y-%m-%d")
        expanded_end = (datetime.strptime(end_date, "%Y-%m-%d") + delta).strftime("%Y-%m-%d")
        extra = get_composite(expanded_start, expanded_end)
        last_used_composite = extra

        update_mask = base.select("Green").mask().Not()
        new_expansion = extra.select("Green").mask().And(update_mask).multiply(offset).rename("expansion_mask")
        expansion_mask = expansion_mask.where(expansion_mask.eq(0).And(new_expansion.neq(0)), new_expansion)

        filled = extra.blend(filled)
        coverage = get_coverage(filled)

    # Attach source scene metadata
    filled = filled.set({'source_ids': last_used_composite.get('source_ids')})
    return filled, expansion_mask

def process_monthly_dswe(start_date, end_date, shapefile_path, max_level, asset_folder,
                         export_dswe=True, export_qc=True, export_composite=True, export_swir2=True):
    """
    Generate and export DSWE composites for each month within the given date range,
    with optional export of DSWE, QC, RGB composite, and SWIR2 products, each to its own fixed subfolder.
    
    Parameters:
    -----------
    start_date : datetime
        Start date for processing
    end_date : datetime
        End date for processing
    shapefile_path : str
        Path to the ROI shapefile
    max_level : int
        Maximum DSWE level to include
    asset_folder : str
        Base folder path for asset exports
    export_dswe : bool, optional (default=True)
        Whether to export DSWE water classification products
    export_qc : bool, optional (default=True)
        Whether to export QC mask products
    export_composite : bool, optional (default=True)
        Whether to export RGB Landsat composite products
    export_swir2 : bool, optional (default=True)
        Whether to export SWIR2 band products
    """
    # Fixed subfolder names
    DSWE_FOLDER = "DSWE_Products"
    COMPOSITE_FOLDER = "Source_LS_Composites"
    QC_FOLDER = "QC_Masks"
    SWIR2_FOLDER = "SWIR_2_LS"
    
    roi = load_roi(shapefile_path)
    current_date = start_date
    
    while current_date <= end_date:
        year, month = current_date.year, current_date.month
        last_day = calendar.monthrange(year, month)[1]
        im_start_date = f"{year}-{month:02d}-01"
        im_end_date = f"{year}-{month:02d}-{last_day:02d}"
        
        try:
            filled_composite, expansion_mask = get_filled_composite_before_dswe(im_start_date, im_end_date, roi)
            dswe_composite = Dswe(filled_composite)
            
            # Check for non-empty result
            if hasattr(dswe_composite, 'size') and dswe_composite.size().getInfo() == 0:
                logging.warning(f"No data available for {year}-{month:02d}.")
            else:
                # Export DSWE
                if export_dswe:
                    export_to_asset(
                        dswe_composite,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{DSWE_FOLDER}",
                        bands=["dswe"],
                        product_name="DSWE"
                    )
                
                # Export QC raster
                if export_qc:
                    export_to_asset(
                        expansion_mask,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{QC_FOLDER}",
                        bands=["expansion_mask"],
                        product_name="QC"
                    )
                
                # Export RGB Landsat composite
                if export_composite:
                    export_to_asset(
                        filled_composite,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{COMPOSITE_FOLDER}",
                        bands=["Blue", "Green", "Red"],
                        product_name="Composite"
                    )
                
                # Export SWIR2 band
                if export_swir2:
                    export_to_asset(
                        filled_composite,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{SWIR2_FOLDER}",
                        bands=["Swir2"],
                        product_name="SWIR2"
                    )
                    
        except Exception as e:
            logging.warning(f"Failed to process {year}-{month:02d}: {e}")
        
        # Move to first of the next month
        current_date = (current_date.replace(day=28) + timedelta(days=4)).replace(day=1)

# Input parameters and export masks as GEE assets

In [4]:
# User-defined parameters
start_date = datetime(1984, 1, 1)
end_date = datetime(2025, 9, 30)
dswe_level = 4
study_area_path = r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\StudyAreas\Delta_UCB\Delta_UCB_WGS84.shp"
gee_asset_output_folder = "projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3"
export_dswe=False
export_qc=False
export_composite=False
export_swir2=True

# Process
process_monthly_dswe(
    start_date, 
    end_date, 
    study_area_path, 
    dswe_level, 
    gee_asset_output_folder,
    export_dswe=export_dswe,
    export_qc=export_qc,
    export_composite=export_composite,
    export_swir2=export_swir2
)

2025-10-26 22:05:05,921 - WARNING - Failed to process 1984-01: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:05:06,371 - WARNING - Failed to process 1984-02: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:05:06,685 - WARNING - Failed to process 1984-03: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:05:06,974 - WARNING - Failed to process 1984-04: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:05:07,215 - WARNING - Failed to process 1984-05: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.

2025-10-26 22:13:37,205 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1987_09...
2025-10-26 22:13:37,458 - WARNING - Failed to process 1987-10: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:13:37,732 - WARNING - Failed to process 1987-11: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:13:37,997 - WARNING - Failed to process 1987-12: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:13:38,172 - WARNING - Failed to process 1988-01: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:13:38,45

2025-10-26 22:24:11,519 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1991_11...
2025-10-26 22:24:11,757 - WARNING - Failed to process 1991-12: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:24:37,186 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1992_01...
2025-10-26 22:25:18,573 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1992_02...
2025-10-26 22:26:20,461 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1992_03...
2025-10-26 22:26:48,049 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1992_04...
2025-10-26 22:27:03,315 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Land

2025-10-26 22:38:28,710 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1996_06...
2025-10-26 22:38:47,959 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1996_07...
2025-10-26 22:39:05,763 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1996_08...
2025-10-26 22:39:17,943 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1996_09...
2025-10-26 22:39:18,222 - WARNING - Failed to process 1996-10: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:39:33,129 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_1996_11...
2025-10-26 22:39:33,415 - WARNING - Failed to process 1996-12: Image.select: Band pattern 'Green' wa

2025-10-26 22:52:46,415 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2001_02...
2025-10-26 22:52:56,880 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2001_03...
2025-10-26 22:53:15,933 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2001_04...
2025-10-26 22:53:16,205 - WARNING - Failed to process 2001-05: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 22:53:30,068 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2001_06...
2025-10-26 22:53:36,101 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2001_07...
2025-10-26 22:53:47,819 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Land

2025-10-26 23:05:00,839 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2005_07...
2025-10-26 23:05:21,507 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2005_08...
2025-10-26 23:05:44,388 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2005_09...
2025-10-26 23:05:45,219 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2005_10...
2025-10-26 23:05:53,914 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2005_11...
2025-10-26 23:05:54,129 - WARNING - Failed to process 2005-12: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 23:05:54,571 - WARNING - Failed to process 2006-01: Image.select: Band pattern 'Green' wa

2025-10-26 23:14:25,414 - WARNING - Failed to process 2009-10: Image.select: Band pattern 'Green' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands
2025-10-26 23:15:08,378 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2009_11...
2025-10-26 23:15:30,800 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2009_12...
2025-10-26 23:15:48,678 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2010_01...
2025-10-26 23:16:13,125 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2010_02...
2025-10-26 23:16:41,435 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2010_03...
2025-10-26 23:16:45,184 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Land

2025-10-26 23:21:22,487 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2013_06...
2025-10-26 23:21:32,736 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2013_07...
2025-10-26 23:21:48,695 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2013_08...
2025-10-26 23:22:10,119 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2013_09...
2025-10-26 23:22:42,933 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2013_10...
2025-10-26 23:23:13,377 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2013_11...
2025-10-26 23:23:48,310 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2013_12...
2025-10-26 23:23:54,486 - INFO - E

2025-10-26 23:37:54,495 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2018_06...
2025-10-26 23:37:55,269 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2018_07...
2025-10-26 23:38:10,176 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2018_08...
2025-10-26 23:38:18,232 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2018_09...
2025-10-26 23:38:28,115 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2018_10...
2025-10-26 23:38:35,668 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2018_11...
2025-10-26 23:39:07,750 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2018_12...
2025-10-26 23:39:41,445 - INFO - E

2025-10-26 23:50:57,185 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2023_06...
2025-10-26 23:51:15,741 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2023_07...
2025-10-26 23:51:32,959 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2023_08...
2025-10-26 23:51:33,754 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2023_09...
2025-10-26 23:51:57,176 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2023_10...
2025-10-26 23:52:11,601 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2023_11...
2025-10-26 23:52:22,314 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/SWIR_2_LS/SWIR2_2023_12...
2025-10-26 23:52:23,048 - INFO - E

## Monitor GEE tasks

In [5]:
# Get list of all running GEE tasks
task_list = ee.batch.Task.list()

# Print task statuses
for task in task_list:
    print(f"Task: {task.status()['description']}, Status: {task.status()['state']}")


Task: SWIR2_2025_09, Status: READY
Task: SWIR2_2025_08, Status: READY
Task: SWIR2_2025_07, Status: READY
Task: SWIR2_2025_06, Status: READY
Task: SWIR2_2025_05, Status: READY
Task: SWIR2_2025_04, Status: READY
Task: SWIR2_2025_03, Status: READY
Task: SWIR2_2025_02, Status: READY
Task: SWIR2_2025_01, Status: READY
Task: SWIR2_2024_12, Status: READY
Task: SWIR2_2024_11, Status: READY
Task: SWIR2_2024_10, Status: READY
Task: SWIR2_2024_09, Status: READY
Task: SWIR2_2024_08, Status: READY
Task: SWIR2_2024_07, Status: READY
Task: SWIR2_2024_06, Status: READY
Task: SWIR2_2024_05, Status: READY
Task: SWIR2_2024_04, Status: READY
Task: SWIR2_2024_03, Status: READY
Task: SWIR2_2024_02, Status: READY
Task: SWIR2_2024_01, Status: READY
Task: SWIR2_2023_12, Status: READY
Task: SWIR2_2023_11, Status: READY
Task: SWIR2_2023_10, Status: READY
Task: SWIR2_2023_09, Status: READY
Task: SWIR2_2023_08, Status: READY
Task: SWIR2_2023_07, Status: READY
Task: SWIR2_2023_06, Status: READY
Task: SWIR2_2023_05,

Task: SWIR2_2001_02, Status: READY
Task: SWIR2_2001_01, Status: READY
Task: SWIR2_2000_12, Status: READY
Task: SWIR2_2000_11, Status: READY
Task: SWIR2_2000_10, Status: READY
Task: SWIR2_2000_09, Status: READY
Task: SWIR2_2000_08, Status: READY
Task: SWIR2_2000_07, Status: READY
Task: SWIR2_2000_06, Status: READY
Task: SWIR2_2000_05, Status: READY
Task: SWIR2_2000_04, Status: READY
Task: SWIR2_2000_02, Status: READY
Task: SWIR2_2000_01, Status: READY
Task: SWIR2_1999_12, Status: READY
Task: SWIR2_1999_11, Status: READY
Task: SWIR2_1999_10, Status: READY
Task: SWIR2_1999_08, Status: READY
Task: SWIR2_1999_06, Status: READY
Task: SWIR2_1999_05, Status: READY
Task: SWIR2_1999_04, Status: READY
Task: SWIR2_1999_03, Status: READY
Task: SWIR2_1999_02, Status: READY
Task: SWIR2_1999_01, Status: READY
Task: SWIR2_1998_12, Status: READY
Task: SWIR2_1998_11, Status: READY
Task: SWIR2_1998_10, Status: READY
Task: SWIR2_1998_09, Status: READY
Task: SWIR2_1998_08, Status: READY
Task: SWIR2_1998_07,

Task: SWIR2_2018_06, Status: FAILED
Task: SWIR2_2018_05, Status: FAILED
Task: SWIR2_2018_04, Status: FAILED
Task: SWIR2_2018_03, Status: FAILED
Task: SWIR2_2018_02, Status: FAILED
Task: SWIR2_2018_01, Status: FAILED
Task: SWIR2_2017_12, Status: FAILED
Task: SWIR2_2017_11, Status: FAILED
Task: SWIR2_2017_10, Status: FAILED
Task: SWIR2_2017_09, Status: FAILED
Task: SWIR2_2017_08, Status: FAILED
Task: SWIR2_2017_07, Status: FAILED
Task: SWIR2_2017_06, Status: FAILED
Task: SWIR2_2017_05, Status: FAILED
Task: SWIR2_2017_04, Status: FAILED
Task: SWIR2_2017_03, Status: FAILED
Task: SWIR2_2017_02, Status: FAILED
Task: SWIR2_2017_01, Status: FAILED
Task: SWIR2_2016_12, Status: FAILED
Task: SWIR2_2016_11, Status: FAILED
Task: SWIR2_2016_10, Status: FAILED
Task: SWIR2_2016_09, Status: FAILED
Task: SWIR2_2016_08, Status: FAILED
Task: SWIR2_2016_07, Status: FAILED
Task: SWIR2_2016_06, Status: FAILED
Task: SWIR2_2016_05, Status: FAILED
Task: SWIR2_2016_04, Status: FAILED
Task: SWIR2_2016_03, Status:

Task: SWIR2_1993_01, Status: FAILED
Task: SWIR2_1992_12, Status: FAILED
Task: SWIR2_1992_11, Status: FAILED
Task: SWIR2_1992_10, Status: FAILED
Task: SWIR2_1992_09, Status: FAILED
Task: SWIR2_1992_08, Status: FAILED
Task: SWIR2_1992_07, Status: FAILED
Task: SWIR2_1992_06, Status: FAILED
Task: SWIR2_1992_05, Status: FAILED
Task: SWIR2_1992_04, Status: FAILED
Task: SWIR2_1992_03, Status: FAILED
Task: SWIR2_1992_02, Status: FAILED
Task: SWIR2_1992_01, Status: FAILED
Task: SWIR2_1991_11, Status: FAILED
Task: SWIR2_1991_10, Status: FAILED
Task: SWIR2_1991_09, Status: FAILED
Task: SWIR2_1991_08, Status: FAILED
Task: SWIR2_1991_07, Status: FAILED
Task: SWIR2_1991_06, Status: FAILED
Task: SWIR2_1991_05, Status: FAILED
Task: SWIR2_1991_04, Status: FAILED
Task: SWIR2_1991_03, Status: FAILED
Task: SWIR2_1991_02, Status: FAILED
Task: SWIR2_1991_01, Status: FAILED
Task: SWIR2_1990_12, Status: FAILED
Task: SWIR2_1990_11, Status: FAILED
Task: SWIR2_1990_10, Status: FAILED
Task: SWIR2_1990_09, Status: